In [1]:
import numpy as np
import pandas as pd
import tqdm 
import cProfile
import lightgbm as lgb
from sklearn.model_selection import GroupKFold, train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import optuna
import logging
from scipy.stats import pearsonr

In [12]:
pheno = "wing"  

data = pd.read_feather(f"data/processed/{pheno}BV.feather")  

In [13]:
data = data[['ID', 'mean_pheno']]

In [14]:
np.var(data)[0]/np.var(data)[1]

0.30804649953559093

In [3]:
#def island_split(pheno:str = "mass") -> pd.DataFrame:

#pheno = "mass"  

#data = pd.read_feather(f"data/processed/{pheno}BV.feather")  
#data = data[["ringnr", "hatchisland"]]


#proceed with only the islands where the values count for hatchisland is greater than 10
#island_counts = data.hatchisland.value_counts()
#island_counts = island_counts[island_counts > 10]
#display(island_counts)
for i, island in enumerate(island_counts.index):
   print(i, island)
   #      fold = data[data.hatchisland == island]["ringnr"]
##    fold_name = f"island_fold_{island}_{pheno}.csv"
#    fold.to_csv(f"data/interim/folds/{fold_name}", index=False)
#    print(f"Island {island} has {len(fold)} samples + {i}")

0 27
1 26
2 28
3 38
4 24
5 20
6 23
7 22


In [33]:
from split import island_split

In [39]:
folds = island_split(pheno="mass")  

In [43]:
len(folds.fold.unique())

8

In [32]:
island_counts



27    719
26    418
28    247
38    153
24    108
20    103
23    100
22     37
Name: hatchisland, dtype: int64

In [22]:
#for island in islands:
#    fold = df[df.hatchisland == island]
#    display(fold.head())

In [29]:
phenotypes = ['mass','tarsus','wing']

for phenotype in phenotypes:
    data = pd.read_feather(f"data/processed/{phenotype}BV.feather")

    mean_pheno = np.round(np.mean(data['mean_pheno']),3)
    var_pheno = np.round(np.var(data['mean_pheno']),3)
    var_BV = np.round(np.var(data['ID']),3)
    corr = pearsonr(data['mean_pheno'],data['ID'])[0]

    print(f"Phenotype: {phenotype}")    
    print(f"Mean of phnotype: {mean_pheno}. Variance of phenotype: {var_pheno}. Variance of BV: {var_BV}. Correlation: {corr}")
    print("\n")

Phenotype: mass
Mean of phnotype: 31.759. Variance of phenotype: 4.647. Variance of BV: 1.434. Correlation: 0.95124696456946


Phenotype: tarsus
Mean of phnotype: 19.466. Variance of phenotype: 0.642. Variance of BV: 0.592. Correlation: 0.987517996538674


Phenotype: wing
Mean of phnotype: 79.714. Variance of phenotype: 5.278. Variance of BV: 1.626. Correlation: 0.739816241885609




In [ ]:
import numpy as np

1000.0

In [15]:
start = np.log10(10)
stop = np.log10(25000)

np.logspace(start, stop, num=12, dtype=int, endpoint=True)

array([   10,    20,    41,    84,   172,   350,   713,  1453,  2959,
        6027, 12275, 24999])

In [22]:
np.arange(1000, 27000, 2500)

array([ 1000,  3500,  6000,  8500, 11000, 13500, 16000, 18500, 21000,
       23500, 26000])